##kaggle competetion https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression

##Data source https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression/data


In [ ]:
#pip install pydicom #installation of pydicom

In [ ]:
#pip install tensorflow-addons #installation of tensorflow-addons

In [ ]:
#from google.colab import drive # Downloading Datasets into Google Drive via Google Colab
import os.path
import os
import cv2
import pydicom
import pandas as pd
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt 
from tqdm.notebook import tqdm 
import seaborn as sns
from sklearn.model_selection import train_test_split 

import tensorflow.keras.utils
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import (Dense, Dropout, Activation, Flatten, Input, BatchNormalization, GlobalAveragePooling2D, Add, Conv2D, AveragePooling2D, 
    LeakyReLU, Concatenate, MaxPooling2D, GlobalMaxPooling2D)
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Nadam
from tensorflow_addons.optimizers import RectifiedAdam
#from tensorflow.keras.applications import EfficientNetB4
#from tensorflow.keras.models import model_from_json


In [ ]:
print(os.listdir("../input/"))
print(os.listdir("../input/efficientnetb4-notoph5"))

In [ ]:
Q = os.listdir('../input/osic-pulmonary-fibrosis-progression')
print('Available dataset partitions: ',Q)

In [ ]:
train = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
train.head()

## Reference code from 
https://www.kaggle.com/miklgr500/linear-decay-based-on-resnet-cnn/execution 

https://www.kaggle.com/purnima29/final-model

In [ ]:
# Converting the column into numericals  
def get_tab(df):
    vector = [(df.Age.values[0] - 30) / 30] 
    
    if df.Sex.values[0] == 'male':
       vector.append(0)
    else:
       vector.append(1)
    
    if df.SmokingStatus.values[0] == 'Never smoked':
        vector.extend([0,0])
    elif df.SmokingStatus.values[0] == 'Ex-smoker':
        vector.extend([1,1])
    elif df.SmokingStatus.values[0] == 'Currently smokes':
        vector.extend([0,1])
    else:
        vector.extend([1,0])
    return np.array(vector) 

A = {} 
TAB = {} 
P = [] 
for i, p in tqdm(enumerate(train.Patient.unique())):
    sub = train.loc[train.Patient == p, :] 
    fvc = sub.FVC.values
    weeks = sub.Weeks.values
    c = np.vstack([weeks, np.ones(len(weeks))]).T
    a, b = np.linalg.lstsq(c, fvc)[0]
    
    A[p] = a
    TAB[p] = get_tab(sub)
    P.append(p)



In [ ]:
train.Patient.values

## CNN for coeff prediction



In [ ]:

def get_img(path):
    d = pydicom.dcmread(path) 
        
    # https://www.geeksforgeeks.org/image-resizing-using-opencv-python/
    # According to model
    return cv2.resize(d.pixel_array / 2**11, (380, 380)) 

## IGenerator 

In [ ]:
class IGenerator(Sequence):
    BAD_ID = ['ID00011637202177653955184', 'ID00052637202186188008618']
    def __init__(self, keys, a, tab, batch_size=16):
        self.keys = [k for k in keys if k not in self.BAD_ID]
        self.a = a
        self.tab = tab
        self.batch_size = batch_size
        
        self.train_data = {}
        self.train_data1 = {}
        self.train_data2= {}

        for p in train.Patient.values:
            self.train_data[p] = os.listdir(f'../input/osic-pulmonary-fibrosis-progression/train/{p}/')
                             
    def __len__(self):
        return 1000
    
    def __getitem__(self, idx):
        x, y, z, w, u, v = [], [], [], [], [], []
        a, tab = [], [] 
        keys = np.random.choice(self.keys, size = self.batch_size)
        for k in keys:
            try:
                i = np.random.choice(self.train_data[k], size=1)[0]
                j = np.random.choice(self.train_data[k], size=1)[0]
                l = np.random.choice(self.train_data[k], size=1)[0]
                m = np.random.choice(self.train_data[k], size=1)[0]
                n = np.random.choice(self.train_data[k], size=1)[0]
                o = np.random.choice(self.train_data[k], size=1)[0]
                # for input image 1
                
                img1 = get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{k}/{i}')
                x.append(img1)
                
                img2 = get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{k}/{j}')
                y.append(img2)
                  
                img3 = get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{k}/{l}')
                z.append(img3)
                
                img4 = get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{k}/{m}')
                w.append(img4)

                img5 = get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{k}/{n}')
                u.append(img5)
                
                img6 = get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{k}/{o}')
                v.append(img6)

                a.append(self.a[k])
                tab.append(self.tab[k])
            except:
                print(k, i)
       
        x,y,z,w,u,v,a,tab = np.array(x), np.array(y), np.array(z), np.array(w), np.array(u), np.array(v), np.array(a), np.array(tab)
        x = np.expand_dims(x, axis=-1)
        y = np.expand_dims(y, axis=-1)
        z = np.expand_dims(z, axis=-1)
        w = np.expand_dims(w, axis=-1)
        u = np.expand_dims(u, axis=-1)
        v = np.expand_dims(v, axis=-1)
        return [x, y, z, w, u, v, tab] , a

# EfficientNetB4

## Pass six images


# 6 input image concats, pass through EfficientNetB4 + tabular data  Model

In [ ]:
# https://www.tensorflow.org/guide/keras/save_and_serialize
model_effnet_B4 = tf.keras.models.load_model("../input/efficientnetb4-notoph5/model_6_image_effnet_B4.h5")

In [ ]:
model_effnet_B4.summary()

In [ ]:
tf.keras.utils.plot_model(model_effnet_B4, 'model_effnet_B4.png', show_shapes=True)

In [ ]:
model_effnet_B4.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0001), loss='mae')

tr_p, vl_p = train_test_split(P, shuffle=True, train_size= 0.8) 

In [ ]:
#https://stackoverflow.com/a/51757115
es = tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=1e-3,patience=4,verbose=1,mode="auto", baseline=None,restore_best_weights=True,)
#my_callbacks = [
    #es,tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    #tf.keras.callbacks.ModelCheckpoint(filepath = '../input/osic-pulmonary-fibrosis-progression/' + 'model_6_image_effnet_B4.h5', verbose=1, save_best_only=True, save_weights_only=False) ]

In [ ]:
history_effnet_B4_Pass_2_images = model_effnet_B4.fit_generator(IGenerator(keys=tr_p, 
                               a = A, 
                               tab = TAB), 
                    steps_per_epoch = 200,
                    validation_data=IGenerator(keys=vl_p, 
                               a = A, 
                               tab = TAB),
                    validation_steps = 20, 
                    callbacks = [es], 
                    epochs=50)

In [ ]:
# summarize history for loss
plt.plot(history_effnet_B4_Pass_2_images.history['loss'])
plt.plot(history_effnet_B4_Pass_2_images.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Generating submission.csv

In [ ]:
def score(fvc_true, fvc_pred, sigma):
    sigma_clip = np.maximum(sigma,70)
    delta = np.abs(fvc_true - fvc_pred)
    delta = np.minimum(delta,1000)
    sq2 = np.sqrt(2)
    metric = (delta/sigma_clip)*sq2  + np.log(sigma_clip*sq2 )
    return np.mean(metric)

In [ ]:
from tqdm.notebook import tqdm

metric = []
for q in tqdm(range(1, 10)):
    m = []
    for p in vl_p:
        x,y,z,w,u,v = [], [], [], [], [], []
        tab = [] 
        
        if p in ['ID00011637202177653955184', 'ID00052637202186188008618']:
            continue
            
        img_set = os.listdir(f'../input/osic-pulmonary-fibrosis-progression/train/{p}/')
        img_set = np.random.choice(img_set, size=20)
        for i in img_set:
            x.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{p}/{i}')) 
            y.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{p}/{i}'))
            z.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{p}/{i}'))
            w.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{p}/{i}'))
            u.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{p}/{i}'))
            v.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{p}/{i}'))
            tab.append(get_tab(train.loc[train.Patient == p, :])) 
    
         
        tab = np.array(tab) 
    
        x = np.expand_dims(x, axis=-1)
        y = np.expand_dims(y, axis=-1)
        z = np.expand_dims(z, axis=-1)
        w = np.expand_dims(w, axis=-1)
        u = np.expand_dims(u, axis=-1)
        v = np.expand_dims(v, axis=-1)
        _a = model_effnet_B4.predict([x,y,z,w,u,v, tab])
        a = np.quantile(_a, q / 10)
        
        percent_true = train.Percent.values[train.Patient == p]
        fvc_true = train.FVC.values[train.Patient == p]
        weeks_true = train.Weeks.values[train.Patient == p]
        
        fvc = a * (weeks_true - weeks_true[0]) + fvc_true[0]
        percent = percent_true[0] - a * abs(weeks_true - weeks_true[0])
        m.append(score(fvc_true, fvc, percent))

    print(np.mean(m))
    metric.append(np.mean(m))

In [ ]:
q = (np.argmin(metric) + 1)/ 10
q

In [ ]:
sub = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv') 
sub.head() 

In [ ]:
test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv') 
test.head()

In [ ]:
A_test, B_test, P_test,W, FVC= {}, {}, {},{},{} 
STD, WEEK = {}, {} 
for p in test.Patient.unique():
    x,y,z,w,u,v = [], [], [], [], [], []
    tab = [] 

   
    img_set = os.listdir(f'../input/osic-pulmonary-fibrosis-progression/test/{p}/')
    img_set = np.random.choice(img_set, size=20)
    for i in img_set:
        x.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/test/{p}/{i}')) 
        y.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/test/{p}/{i}'))
        z.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/test/{p}/{i}')) 
        w.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/test/{p}/{i}'))
        u.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/test/{p}/{i}')) 
        v.append(get_img(f'../input/osic-pulmonary-fibrosis-progression/test/{p}/{i}'))

        tab.append(get_tab(train.loc[train.Patient == p, :])) 
    tab = np.array(tab) 
            
    x = np.expand_dims(x, axis=-1) 
    y = np.expand_dims(y, axis=-1)
    z = np.expand_dims(z, axis=-1) 
    w = np.expand_dims(w, axis=-1)
    u = np.expand_dims(u, axis=-1) 
    v = np.expand_dims(v, axis=-1) 

    _a = model_effnet_B4.predict([x,y,z,w,u,v, tab]) 
    a = np.quantile(_a, q)
    A_test[p] = a
    B_test[p] = test.FVC.values[test.Patient == p] - a*test.Weeks.values[test.Patient == p]
    P_test[p] = test.Percent.values[test.Patient == p] 
    WEEK[p] = test.Weeks.values[test.Patient == p]

In [ ]:
for k in sub.Patient_Week.values:
    p, w = k.split('_')
    w = int(w) 
    
    fvc = A_test[p] * w + B_test[p]
    sub.loc[sub.Patient_Week == k, 'FVC'] = fvc
    sub.loc[sub.Patient_Week == k, 'Confidence'] = (
        P_test[p] - A_test[p] * abs(WEEK[p] - w) 
) 
sub.head()

In [ ]:
sub[["Patient_Week","FVC","Confidence"]].to_csv("submission.csv", index=False)

In [ ]:
sub

In [ ]:
#
#effnet_B4_model = tensorflow.keras.models.load_model("../input/efficientnetb4-notoph5/model_6_image_effnet_B4.h5")

In [ ]:
## https://www.kaggle.com/rtatman/download-a-csv-file-from-a-kernel

# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe
df = sub

# create a link to download the dataframe
create_download_link(df)

# ↓ ↓ ↓  Yay, download link! ↓ ↓ ↓ 
